In [ ]:
%load_ext autoreload
%autoreload 2

!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -y --offline
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -y --offline
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -y --offline
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -y --offline
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -y --offline
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -y --offline

## MMDetection compatible torch installation
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

## Compatible Cuda Toolkit installation
!mkdir -p /kaggle/tmp && cp /kaggle/input/pytorch-170-cuda-toolkit-110221/cudatoolkit-11.0.221-h6bb024c_0 /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 && conda install /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 -y --offline

## MMDetection Offline Installation
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e . --no-deps
%cd /kaggle/working/

In [ ]:
# ! cp -f ../input/bug-fix/cascade_roi_head.py ./mmdetection/mmdet/models/roi_heads/cascade_roi_head.py

In [ ]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import pydicom
import skimage.io
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
import sys
sys.path.insert(0, "./mmdetection")
from mmcv import Config
from mmdet.apis import init_detector, inference_detector

cfg_path = '../input/siim-config/cascade_rcnn.py'
checkpoint = '../input/siimcovid19detection/cascade_rcnn.pth'
cfg = Config.fromfile(cfg_path)
cascade_net = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
IMG_SIZE = 214

def build_efficient_net():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
        EfficientNetB0(include_top=True, weights=None, classes=4)
    ])
    return model

efficient_net = build_efficient_net()
efficient_net.load_weights('../input/efficientnet/efficient.h5')

In [ ]:
submission_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_paths = []
files = glob.glob(f"../input/siim-covid19-detection/test/*/*/*")
for file in files:
    test_paths.append(file)
# test_paths = test_paths[:20]
dicts = []
for path in tqdm(test_paths):
    file = pydicom.dcmread(path)
    image_id = path[-16:-4] + '_image'
    study_id = path.split('/')[4] + '_study'
    width = file.Columns
    height = file.Rows
    dicts.append({'image_id': image_id, 'study_id': study_id, 'path': path, 'width': width, 'height': height})
test_df = pd.DataFrame(dicts)

In [ ]:
! rm -rf test_convert
! mkdir test_convert

In [ ]:
TEST_DIR = "./test_convert/"
to_grayscale = lambda x: np.uint8(x / x.max() * 255)
SIZE = 512

def save_dicom(image, name, path=TEST_DIR):
    image = to_grayscale(image)
    image = resize(image, (SIZE, SIZE))
    image = to_grayscale(image)
    path = path + name
    skimage.io.imsave(path, image)
    
def save_test_image(dicom_path):
    name = dicom_path[-16:-4] + '_512.png'
    image = pydicom.dcmread(dicom_path).pixel_array
    save_dicom(image, name, path=TEST_DIR)
    
for path in tqdm(test_paths):
    save_test_image(path)

In [ ]:
classes = ['negative', 'typical', 'indeterminate', 'atypical']

def predict_type(path):
    image = skimage.io.imread(path)
    image = resize(image, (224, 224))
    image = np.stack((image, image, image), axis=2)
    image = image.reshape(1, 224, 224, 3)
    index = efficient_net(image).numpy().argmax()
    label = classes[index]
    return label

def predict_row(row):
    name = row['image_id'][:-6] + '_512.png'
    path = TEST_DIR + name
    result = inference_detector(cascade_net, path)
    label = predict_type(path)
    boxes = result[0][:, :4]
    scores = result[0][:, 4]
    output = ''
    cnt = 0
    for index in range(len(result[0])):
        if scores[index] < 0.5:
            continue
        cnt += 1
        prediction = 'opacity ' + "{:.2f}".format(scores[index]) + ' '
        box = boxes[index]
        h = row['height']
        w = row['width']
        x_min = box[0] / 512 * w
        y_min = box[1] / 512 * h
        x_max = box[2] / 512 * w
        y_max = box[3] / 512 * h
        for n in [x_min, y_min, x_max, y_max]:
            prediction += "{:.2f}".format(n) + ' '
        output += prediction
    submission_df.loc[submission_df['id']==row['study_id'], ['PredictionString',]] = label + ' 1 0 0 1 1'
    if cnt > 0:
        submission_df.loc[submission_df['id']==row['image_id'], ['PredictionString',]] = output

In [ ]:
for _, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    predict_row(row)
submission_df.to_csv('submission.csv', index=False)

In [ ]:
!rm -rf /kaggle/working/mmdetection
!rm -rf /kaggle/working/test_convert